In [ ]:
#!mkdir -p drive
#

In [ ]:
import os
# 此处为google drive中的文件路径,drive为之前指定的工作根目录，要加上
#os.chdir("drive/My Drive/2023kd")

In [ ]:
import pandas as pd
df=pd.read_csv('mobilegt-unifeaturedata-labelnumber.csv')
df=df.drop(['Unnamed: 0'],axis=1)
groupby_data=df.groupby(['25'])['25'].count()   #标签类别分类计数
df

In [4]:
df_1 = df.drop(df[(df['25']==1) | (df['25']==5)| (df['25']==8)| (df['25']==9)| (df['25']==10)].index)
#df_1.to_csv('mobilegt-unifeaturedata-minority.csv',index = False)

In [32]:
x=df_1.iloc[:,:-1]
y=df_1.iloc[:,-1]
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(x,y,stratify=y,random_state=40,test_size=0.4)
y_test = list(y_test)
KNC = KNeighborsClassifier(n_neighbors=10).fit(X_train,y_train)
y_pred = KNC.predict(X_test)

In [6]:
mino_label = [0,2,3,4,6,7,11]
y_pred_new = {}
y_test_new = {}

for l in mino_lable:
  for i in range(len(y_test)):
    if l in y_pred_new:
      if y_test[i] == l:
        y_pred_new[l].append(y_pred[i])
        y_test_new[l].append(y_test[i])
      else:
        y_pred_new[y_test[i]].append(y_pred[i])
        y_test_new[y_test[i]].append(y_test[i])
    else:
      if y_test[i] == l:
        y_pred_new[l] = []
        y_test_new[l] = []
        y_pred_new[l].append(y_pred[i])
        y_test_new[l].append(y_test[i])
      else:
        y_pred_new[y_test[i]].append(y_pred[i])
        y_test_new[y_test[i]].append(y_test[i])

for i in range(len(y_test)):
  if y_test[i] in mino_label:
    # print (y_test[i])
    # print ('1111111111111111111111')
    if y_test[i] in y_pred_new:
      y_pred_new[y_test[i]].append(y_pred[i])
      y_test_new[y_test[i]].append(y_test[i])
    else:
      y_pred_new[y_test[i]]=[]
      y_test_new[y_test[i]]=[]
      y_pred_new[y_test[i]].append(y_pred[i])
      y_test_new[y_test[i]].append(y_test[i])

In [7]:
print(len(y_pred_new))

7


In [ ]:
from sklearn import metrics
for i in mino_label:
  label_true=y_test_new[i]
  label_pred=y_pred_new[i]
  print(type(label_pred))
  ARI = metrics.adjusted_rand_score(label_true,label_pred)
  FM = metrics.fowlkes_mallows_score(label_true,label_pred)

  print(ARI,FM)

<class 'list'>
0.0 0.46191525636568714
<class 'list'>
0.0 0.7796081664696198
<class 'list'>
0.0 0.5682068802298305
<class 'list'>
0.0 0.6180694036829117
<class 'list'>
0.0 0.6254329187932717
<class 'list'>
0.0 0.6085795988997669
<class 'list'>
0.0 0.5641328520654737


In [30]:
import math
import numpy as np
from sklearn import metrics
def NMI(A,B):
    # 样本点数
    total = len(A)
    A_ids = set(A)
    B_ids = set(B)
    # 互信息计算
    MI = 0
    eps = 1.4e-45
    for idA in A_ids:
        for idB in B_ids:
            idAOccur = np.where(A==idA)    # 输出满足条件的元素的下标
            #idABOccur = np.intersect1d(idAOccur,idBOccur)   # Find the intersection of two arrays.
            px = 1.0*len(idAOccur[0])/total
            py = 1.0*len(B)/total
            pxy = 1.0*len(idAOccur[0])/total
            print(px,py,pxy)
            MI = MI + pxy*math.log(pxy/(px*py)+eps,2)
            print(MI)
    # 标准化互信息
    Hx = 0
    for idA in A_ids:
        idAOccurCount = 1.0*len(np.where(A==idA)[0])
        Hx = Hx - (idAOccurCount/total)*math.log(idAOccurCount/total+eps,2)
        Hy = 0
    for idB in B_ids:
        idBOccurCount = 1.0*len(np.where(B==idB)[0])
        Hy = Hy - (idBOccurCount/total)*math.log(idBOccurCount/total+eps,2)
    MIhat = 2.0*MI/(Hx+Hy)
    return MIhat

In [ ]:
from sklearn import metrics
for i in mino_label:
  label_true=y_test_new[i]
  label_pred=y_pred_new[i]
  print(label_true)
  print(NMI(label_pred,label_true))

存储并获得少数类难分类样本过采样数据集：

In [34]:
df_2 = df.drop(df[(df['25']==1) | (df['25']==2) | (df['25']==5) | (df['25']==8)| (df['25']==9)| (df['25']==10)].index)

In [ ]:
df_2

In [36]:
x=df_2.iloc[:,:-1]
y=df_2.iloc[:,-1]

In [ ]:
!pip install imblearn

In [43]:
from imblearn.combine import SMOTETomek
from collections import Counter
kos = SMOTETomek(random_state=40,sampling_strategy='all')  # 综合采样
x_kos,y_kos = kos.fit_resample(x,y)
print('综合采样后，训练集 y_kos 中的分类情况：{}'.format(Counter(y_kos)))

综合采样后，训练集 y_kos 中的分类情况：Counter({4.0: 5740, 6.0: 5669, 0.0: 5628, 11.0: 5340, 3.0: 5337, 7.0: 5196})


In [ ]:
print(type(x_kos))
print(y_kos)
df_smote = pd.concat([x_kos,y_kos],axis=1)
#df_smote.to_csv('mobile-minority-smote.csv',index = None,encoding = 'utf8')

将合成好的和原来的一起拼接！

In [56]:
df_3 = df.drop(df[(df['25']==0) | (df['25']==3) | (df['25']==4) | (df['25']==6)| (df['25']==7)| (df['25']==11)].index)
df_all = pd.concat([df_smote,df_3],axis=0)
df_all.to_csv('mobile-all-smote.csv',index = None,encoding = 'utf8')

In [45]:
from imblearn.over_sampling import KMeansSMOTE
from collections import Counter
kos = KMeansSMOTE(random_state=40,sampling_strategy='all',k_neighbors=5)  # 综合采样
x_kos1,y_kos1 = kos.fit_resample(x,y)
print('综合采样后，训练集 y_kos 中的分类情况：{}'.format(Counter(y_kos1)))

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


RuntimeError: ignored